## File Preparations

In [51]:
# Dependencies
import pandas as pd
import csv

In [52]:
# Read in CSV
csvpath = "Resources/purchase_data.csv"
pymoli_df = pd.read_csv(csvpath)
pymoli_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## General Numbers

#### Purchasing Analysis - Overall

In [53]:
# Total player count
player_count = len(pymoli_df["SN"].unique())
player_count

576

In [54]:
# Initialize Purchasing Analysis Dictionary
purchasing_analysis_overall = {}

In [55]:
# Number of Unique Items
purchasing_analysis_overall["Unique Items Available"] = [len(pymoli_df["Item Name"].unique())]
#### value is bracketed so it will act as index when creating the dataframe

In [56]:
# Average Purchase Price
purchasing_analysis_overall["Average Price"] = "${:,.2f}".format(round(pymoli_df["Price"].mean(),2))

In [57]:
# Total Number of Purchases
purchasing_analysis_overall["Total Purchases"] = pymoli_df["Purchase ID"].count()

In [58]:
# Total Revenue
purchasing_analysis_overall["Total Revenue"] = "${:,.2f}".format(pymoli_df["Price"].sum())

In [59]:
# Configure and output table
purchasing_analysis_overall_df = pd.DataFrame(purchasing_analysis_overall)
purchasing_analysis_overall_df.head()

,Unique Items Available,Average Price,Total Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


#### Gender Demographics

In [60]:
# Initialize gender dictionaries and individual subset dataframes
male = {}
male_df = pymoli_df.loc[pymoli_df["Gender"] == "Male", :]

female = {}
female_df = pymoli_df.loc[pymoli_df["Gender"] == "Female", :]

other = {}
other_df = pymoli_df.loc[pymoli_df["Gender"] == "Other / Non-Disclosed", :]

In [76]:
# Male, Female, Other/Non-disclosed
# - Count
# - Percentage

# male
male[""] = ["Male"] # will act as row index for "male" in gender_demographics_df
male["Total Count"] = len(male_df["SN"].unique())
male["Percent of Players"] = "{:.2%}".format(male["Total Count"]/player_count)

# female
female[""] = ["Female"] # will act as row index for "female" in gender_demographics_df
female["Total Count"] = len(female_df["SN"].unique())
female["Percent of Players"] = "{:.2%}".format(female["Total Count"]/player_count)

# other
other[""] = ["Other / Non-Disclosed"] # will act as row index for "other" in gender_demographics_df
other["Total Count"] = len(other_df["SN"].unique())
other["Percent of Players"] = "{:.2%}".format(other["Total Count"]/player_count)

In [77]:
# Configure and output table
gender_dicts = [male, female, other]
gender_dfs = [pd.DataFrame(x) for x in gender_dicts]

In [80]:

gender_demographics_df = pd.concat(gender_dfs).set_index("")
gender_demographics_df

,Total Count,Percent of Players
,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


#### Purchasing Analysis - Gender

In [ ]:
# Male, Female, Other/Non-disclosed
# - Purchase Count
# - Total Purchase Value
# - Average Purchase Price
# - Average Purchase Total per Person

# male
male_purchase_count = male_df["Price"].count()
male_total_purchase = male_df["Price"].sum()
male_avg_purchase = "${:,.2f}".format(male_df["Price"].mean())
male_avg_per_person = "${:,.2f}".format(male_total_purchase/male_player_count)
male_total_purchase = "${:,.2f}".format(male_total_purchase) # format after making necessary calculations

# female
female_purchase_count = female_df["Price"].count()
female_total_purchase = female_df["Price"].sum()
female_avg_purchase = "${:,.2f}".format(female_df["Price"].mean())
female_avg_per_person = "${:,.2f}".format(female_total_purchase/female_player_count)
female_total_purchase = "${:,.2f}".format(female_total_purchase) # format after making necessary calculations

# other
other_purchase_count = other_df["Price"].count()
other_total_purchase = other_df["Price"].sum()
other_avg_purchase = "${:,.2f}".format(other_df["Price"].mean())
other_avg_per_person = "${:,.2f}".format(other_total_purchase/other_player_count)
other_total_purchase = "${:,.2f}".format(other_total_purchase) # format after making necessary calculations

In [ ]:
# Configure and output table
purchasing_analysis_gender_df = pd.DataFrame({
    "":["Male", "Female", "Other"],
    "Transactions": [male_purchase_count, female_purchase_count, other_purchase_count],
    "Total Purchase Amount": [male_total_purchase, female_total_purchase, other_total_purchase],
    "Average Purchase Amount": [male_avg_purchase, female_avg_purchase, other_avg_purchase],
    "Average Purchase Amount Per Person": [male_avg_per_person, female_avg_per_person, other_avg_per_person]
}).set_index("")
purchasing_analysis_gender_df

# STOP - Go back!
1. Make each gender a dictionary
1. Get rid of the extraneous variables
1. Clean up the data frame creation
---
---

#### Age Demographics

In [ ]:
# Age Bins
# - Purchase Count
# - Average Purchase Price
# - Total Purchase Value
# - Average Purchase Total per Person

## Superlatives

#### Top Spenders

In [ ]:
# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
# - SN
# - Purchase Count
# - Average Purchase Price
# - Total Purchase Value

#### Most Popular Items

In [ ]:
# Identify the 5 most popular items by purchase count, then list (in a table):
# - Item ID
# - Item Name
# - Purchase Count
# - Item Price
# - Total Purchase Value

#### Most Profitable Items

In [ ]:
# Identify the 5 most profitable items by total purchase value, then list (in a table):
# - Item ID
# - Item Name
# - Purchase Count
# - Item Price
# - Total Purchase Value